In [54]:
import os
import pandas as pd
import sqlite3
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
import numpy as np

In [55]:
# Define the SQLite database path
db_path = '/Users/Shravanthi/TAADS/courses01.sqlite'

# Define the folder containing the dataset files
data_folder = '/Users/Shravanthi/TAADS/datasets'  

In [56]:
# Function to clean a dataset
def clean_dataset(df):
    # Fill missing values
    df.fillna("Not specified", inplace=True)

    # Standardize column names
    df.columns = [col.lower().replace(" ", "_") for col in df.columns]

    # Handle invalid prices or negative values if relevant columns exist
    if 'price' in df.columns and 'num_subscribers' in df.columns:
        df = df[(df['price'] >= 0) & (df['num_subscribers'] >= 0)]
    return df

In [57]:
# Function to add a dataset to the SQLite database
def add_to_sqlite(df, table_name, db_path):
    conn = sqlite3.connect(db_path)
    df.to_sql(table_name, conn, if_exists='replace', index=False)
    conn.close()

In [58]:
# Loop through all files in the data folder
for file_name in os.listdir(data_folder):
    file_path = os.path.join(data_folder, file_name)

    # Read the dataset based on file extension
    if file_name.endswith('.csv'):
        df = pd.read_csv(file_path)
    else:
        print(f"Unsupported file format: {file_name}")
        continue

    # Clean the dataset
    df_cleaned = clean_dataset(df)

    # Generate a table name based on the file name
    table_name = os.path.splitext(file_name)[0].lower().replace(" ", "_")

    # Add the cleaned dataset to the SQLite database
    add_to_sqlite(df_cleaned, table_name, db_path)

    print(f"Added {file_name} as table {table_name} in SQLite database.")

# Verify the tables in the SQLite database
conn = sqlite3.connect(db_path)
created_tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
conn.close()

print("Tables in the database:", created_tables)

Added EdX.csv as table edx in SQLite database.
Added coursera.csv as table coursera in SQLite database.


/var/folders/gc/4knrk_3912gc2lfc0skhzgvh0000gn/T/ipykernel_82403/846106797.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Not specified' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna("Not specified", inplace=True)


Added Coursera_2.csv as table coursera_2 in SQLite database.
Unsupported file format: merged_courses.sqlite
Added udemy_courses.csv as table udemy_courses in SQLite database.
Added CoursesData.csv as table coursesdata in SQLite database.
Tables in the database: [('cleaned_courses',), ('edx',), ('coursera',), ('coursera_2',), ('udemy_courses',), ('coursesdata',)]


In [59]:
# Function to load data from all tables in SQLite database
def load_data_from_all_tables(db_path):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Get all table names from the database
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()

    documents = []
    for table_name in tables:
        table_name = table_name[0]  # Extract table name from tuple
        print(f"Loading data from table: {table_name}")

        # Fetch all rows from the table
        cursor.execute(f"SELECT * FROM {table_name}")
        rows = cursor.fetchall()

        # Combine rows into Document objects
        # Assuming the content starts at index 1 to skip a potential primary key column
        for row in rows:
            page_content = " ".join(map(str, row[1:]))  # Combine all columns except the first
            documents.append(Document(page_content=page_content))

    conn.close()
    return documents

In [ ]:
# Define the database path
db_path = "/Users/Shravanthi/TAADS/courses01.sqlite"

# Load data from all tables
documents = load_data_from_all_tables(db_path)

Loading data from table: cleaned_courses
Loading data from table: edx
Loading data from table: coursera
Loading data from table: coursera_2
Loading data from table: udemy_courses
Loading data from table: coursesdata


In [61]:
# Split documents into manageable chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=500)
final_documents = text_splitter.split_documents(documents)

# Verify loading and splitting
print(final_documents[0])
print("Total number of document chunks:", len(final_documents))

▪ Self-study There are no formal requirements. Nieuwenhuis, P. / Wells, P.: Global Automotive Industry, John Wiley & Sons, 2015\nMunson, C.: The Supply Chain Management Casebook, FT Press, 2013Diehlmann, J. / Häcker, J.: Automotive Management, 2nd ed., Oldenbourg Verlag, 2013 Myerson, P. A.: Lean and Technology: Working Hand in Hand to Enable and Energize Your Global Supply Chain, Pearson Education, 2017F urther references will be given during the classes. This course is in particular applicable to the following Master programmes: International Business and Economics (M.A.; “IBE”).
▪ Readings and exam preparation: 54 6  Comprehensive written examination, 90 minutes (100%) First academic year Summer 1 Semester Elective English b'`\xe4\\<\xf6\xb4\x00=\xad▪ Self-study There are no formal requirements. Nieuwenhuis, P. / Wells, P.: Global Automotive Industry, John Wiley & Sons, 2015\nMunson, C.: The Supply Chain Management Casebook, FT Press, 2013Diehlmann, J. / Häcker, J.: Automotive Man

In [62]:
# Initialize embeddings using HuggingFace
huggingface_embeddings = HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

# Example embedding for verification
embedding_example = np.array(huggingface_embeddings.embed_query(final_documents[0].page_content))
print("Embedding vector for the first document chunk:", embedding_example)
print("Shape of the embedding vector:", embedding_example.shape)

Embedding vector for the first document chunk: [-1.20745432e-02  7.87301734e-02  3.50556932e-02 -3.51749198e-03
  4.15791199e-02  1.76234301e-02 -2.72782776e-03  7.31812716e-02
 -7.47286752e-02 -6.48280084e-02  3.67245451e-02 -3.15500535e-02
  1.89472776e-04 -8.75324756e-03  1.52375121e-02  1.68769844e-02
  3.09608914e-02  6.90320181e-03  5.20508289e-02 -7.78147578e-02
 -1.28828706e-02 -4.48709019e-02 -5.97062148e-03 -1.52579965e-02
  5.33557013e-02  1.23097170e-02  2.62966603e-02  2.56084800e-02
 -2.69088540e-02 -1.89657047e-01 -5.13429195e-02  2.75295712e-02
  1.83006283e-02  6.51086867e-02  9.47162881e-03  3.43764909e-02
 -7.25370049e-02 -9.51696001e-03  9.83149093e-03 -5.51961847e-02
 -4.87993285e-03  4.15199064e-03  1.42299077e-02 -7.33447745e-02
  1.63218100e-02 -3.56145836e-02  3.48471384e-03 -2.20933761e-02
 -1.34163564e-02 -6.70475885e-02 -1.20901065e-02 -5.57899065e-02
  1.56254657e-02 -4.86150086e-02 -1.90979540e-02  4.46648337e-02
  2.90481038e-02  1.15750451e-02 -1.9045153

In [63]:
# Create FAISS VectorStore for similarity search
vectorstore = FAISS.from_documents(final_documents, huggingface_embeddings)

# Example Query Using Similarity Search
query = "Tell me about Industrial Economics"
relevant_documents = vectorstore.similarity_search(query)

# Display content of the most relevant document found
print("Most relevant document content:\n", relevant_documents[0].page_content)

# Set up the retriever with similarity search configuration
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 1})
print("Retriever set up successfully:", retriever)

Most relevant document content:
 National Research University Higher School of Economics Advanced 4.9 https://www.coursera.org/learn/industrial-organization Industrial Organization is the area of economics that studies the markets as institutions, the state of competition and strategic interaction among firms, the industrial policy and the business decisions firms make within the market framework. The course looks at the markets from three different perspectives: the economic theory, the applied business perspective and the institutional and legal perspective. The focus of the course is split equally between the economic theory and business perspective but there is a significant legal component incorporated in various topics. The course includes economic modeling, game theory, numerous real life examples and several case studies. We explore interesting topics of market organization such as negotiations, antitrust, networks, platforms, electronic markets, intellectual property, business

In [64]:
# Hugging Face API setup (replace with your Hugging Face API token)
os.environ['HUGGINGFACEHUB_API_TOKEN'] = "Your Hugging Face Token"

# Initialize Hugging Face model for question-answering
from langchain_community.llms import HuggingFaceHub

hf = HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-Instruct-v0.3",
    model_kwargs={"temperature": 0.8, "max_length": 300}
)

In [65]:
# Execute a query to get a response
response = hf.invoke(query)
print("Response from Hugging Face model:\n", response)

prompt_template = """
Your answer must be based solely on the context provided below. Do not include any unrelated information.

Context:
{context}

Question:
{question}

Answer:
"""

Response from Hugging Face model:
 Tell me about Industrial Economics, Inc.
Industrial Economics, Inc. (IE) is a leading economics consulting firm, with offices in Boston, Washington, D.C., Brussels, and Singapore, and more than 100 staff members. IE’s practice areas include antitrust, regulation, competition policy, energy, environmental economics, litigation support, and international trade.

IE was founded in 1985 by faculty members of the Massachusetts Institute of Technology with a mission to provide clients with rigorous, independent, and actionable economic analysis. We provide consulting services to major corporations, law firms, and government agencies. We have testified before courts and regulatory agencies, including the Federal Trade Commission, the Department of Justice, the European Commission, the U.K. Competition and Markets Authority, and the China National Development and Reform Commission. Our clients have included American Airlines, AT&T, Bayer, Boeing, Boston Scien

In [66]:
from langchain import PromptTemplate
prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

from langchain.chains import RetrievalQA
retrievalQA = RetrievalQA.from_chain_type(
    llm=hf,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}
)

In [67]:
# Backend function
def search_query(query):
    try:
        # Call the QA chain with the query
        result = retrievalQA.invoke({"query": query})

        # Directly extract the response content
        full_response = result['result']  # Retrieve the generated result from the chain

        # Find the "Answer:" portion and isolate the actual answer
        if "Answer:" in full_response:
            answer_start = full_response.find("Answer:") + len("Answer:")
            answer = full_response[answer_start:].strip()
        else:
            # Fallback if "Answer:" label is missing
            answer = full_response.strip()

        # Format the output with Question and Answer only
        return f"Question:\n{query}\n\nAnswer:\n{answer}"
    except Exception as e:
        # Handle errors gracefully
        return f"An error occurred: {e}"

# Gradio UI with enhanced appearance
import gradio as gr

with gr.Blocks() as demo:
    # Title Section with Softer Background
    with gr.Row(elem_id="title-section"):
        with gr.Column():
            gr.Markdown(
                """
                <div style="font-size: 32px; font-family: 'Montserrat', sans-serif; color: white; text-align: center;">
                COURSE HUNTER
                </div>
                <div style="font-size: 18px; font-family: 'Lato', sans-serif; color: white; text-align: center;">
                <em>Your gateway to smarter learning—start exploring today!!</em>
                </div>
                """,
                elem_id="title",
            )

    # Main Functionality Section
    with gr.Row():
        with gr.Column(scale=1):
            query = gr.Textbox(label="I’m here to help—just ask!", placeholder="Enter your search term")
            search_button = gr.Button("Search")
        with gr.Column(scale=2):
            output = gr.Textbox(label="Results", interactive=False)

    # Connect the search button with the search function
    search_button.click(fn=search_query, inputs=[query], outputs=output)

    # Add CSS for enhanced styling
    demo.css = """
        body {
            background-color: #F7F9FC !important;
            color: #F8F8F8 !important;
            font-family: 'Lato', sans-serif;
        }
        #title-section {
            background-color: #5D6D7E !important;
            padding: 20px;
            border-radius: 8px;
            margin-bottom: 20px;
        }
        .gradio-input, .gradio-output {
            border: 1px solid #DADFE3;
            background-color: white;
            color: #555555;
            border-radius: 5px;
            padding: 8px;
        }
        button {
            background-color: #3498DB !important;
            color: white !important;
            border: none;
            padding: 8px 16px;
            border-radius: 5px;
            font-size: 16px;
            cursor: pointer;
        }
        button:hover {
            background-color: #2980B9 !important;
        }
        .gr-row, .gr-column {
            margin: 8px 0;
        }
           /* Add shaded outer frame */
        .gradio-container {
            border: 3px solid #D3D3D3;
            border-radius: 10px;
            padding: 20px;
            margin: 40px auto;
            box-shadow: 0px 8px 16px rgba(0, 0, 0, 0.1); /* Adds a subtle shadow effect */
            max-width: 800px;
            background-color: #FFFFFF;
        }
    """

# Launch the interface
demo.launch()

* Running on local URL:  http://127.0.0.1:7890

To create a public link, set `share=True` in `launch()`.
